In [35]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %load_ext autoreload
%autoreload 2

from Model.Core.model import make_model
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from dotenv import load_dotenv
load_dotenv(override=True)

Python-dotenv could not parse statement starting at line 3


True

In [36]:
BATCH_SIZE = int(os.getenv("BATCH_SIZE"))
IMG_SIZE = eval(os.getenv("IMG_SIZE"))
path_full_dataset = os.getenv("DatesetPath")

def load_dataset(path, image_size,batch_size,shuffle=True ):
    train_dataset = None
    try:
        train_dataset = tf.keras.utils.image_dataset_from_directory(path,
                                                                    shuffle=shuffle,
                                                                    batch_size=batch_size,
                                                                    image_size=image_size)
    except:
        raise
    return train_dataset

In [37]:
full_version_Dataset = load_dataset(path_full_dataset, IMG_SIZE, BATCH_SIZE)

Found 400 files belonging to 2 classes.


In [38]:
IMG_SHAPE = IMG_SIZE + (3,)
print(IMG_SHAPE)

(216, 384, 3)


In [40]:
model = make_model(batch=BATCH_SIZE, shape=IMG_SHAPE)

85622784/94765736 [==========================>...] - ETA: 2s

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host